前回と同じリポジトリ(nlp)にjupyterを格納してください。

In [178]:
!pip install gensim mecab-python3 unidic-lite pandas nltk scikit-learn

## 問題4-1. Bag Of Words (テキスト外), mecabによる品詞分解
* Bag Of Wordsについて調べて理解したことを記載してください。
* 以下のテキストを利用してBag Of Wordsを算出してください。そのためにmecabを用いて'名詞', '動詞', '形容詞'のみを抽出し計測してください。
```
text = """
私は昨日、新しい本を買いました。
その本はとても面白くて、一気に読み終えました。
読書は私の大好きな趣味の一つです。
新しい知識を得られるので、毎日少しずつ本を読むようにしています。
"""
```


・Bag Of Words : 単語ごとに出現回数を集計したもの

In [179]:
!pip install mecab-python3
!pip install unidic-lite

In [180]:
from sklearn.feature_extraction.text import CountVectorizer
import MeCab

In [181]:
# MeCabで名詞、動詞、形容詞のみを対象としてBag Of Wordsを算出
mecab_tagger = MeCab.Tagger()

text = """
私は昨日、新しい本を買いました。
その本はとても面白くて、一気に読み終えました。
読書は私の大好きな趣味の一つです。
新しい知識を得られるので、毎日少しずつ本を読むようにしています。
"""

node = mecab_tagger.parseToNode(text)
count_dict = {}
target_list = ['名詞', '動詞', '形容詞']

while node:
    word = node.surface
    hinshi = node.feature.split(',')[0]
    if word in count_dict.keys() and hinshi in target_list:
        count_dict[word] += 1
    elif hinshi in target_list:
        count_dict[word] = 1
    else:
        pass
    node = node.next

word_counts = sorted(count_dict.items(), key=lambda x:x[1], reverse=True)
word_counts


[('本', 3),
 ('新しい', 2),
 ('昨日', 1),
 ('買い', 1),
 ('面白く', 1),
 ('一気', 1),
 ('読み', 1),
 ('終え', 1),
 ('読書', 1),
 ('趣味', 1),
 ('一', 1),
 ('知識', 1),
 ('得', 1),
 ('毎日', 1),
 ('読む', 1),
 ('し', 1),
 ('い', 1)]

## 問題4-2. N-Gram (テキスト外)
1. N-Gramについて調査してください。
2. 以下の文書でBigramを算出してください。

```
text = "自然言語処理はとても面白い分野です。多くの応用があり、可能性は無限です。"
```
3. ヒント: ngram計算には形態素解析で分解する必要があります。
4. ヒント: Bigramはfrom nltk.util import ngramsを利用すると簡単にできます。


* Unigram (1-gram): 単一の単語（または文字）。
    例: "I love NLP" では、Unigram は ["I", "love", "NLP"] です。

* Bigram (2-gram): 連続する2つの単語（または文字）の組み合わせ。
    例: "I love NLP" では、Bigram は ["I love", "love NLP"] です。

* Trigram (3-gram): 連続する3つの単語（または文字）の組み合わせ。
    例: "I love NLP" では、Trigram は ["I love NLP"] です。

N-Gram: 連続するn個の単語や文字のまとまり
単語同士の繋がりが分かりやすいという特徴がある

In [182]:
# 形態素解析による分解
import MeCab

mecab_tagger = MeCab.Tagger()
text = "自然言語処理はとても面白い分野です。多くの応用があり、可能性は無限です。"

node = mecab_tagger.parseToNode(text)
word_list = []

while node:
    word_list.append(node.surface)
    node = node.next

print(word_list)

['', '自然', '言語', '処理', 'は', 'とても', '面白い', '分野', 'です', '。', '多く', 'の', '応用', 'が', 'あり', '、', '可能', '性', 'は', '無限', 'です', '。', '']


In [183]:
# Biogramの作成
from nltk.util import ngrams
list(ngrams((word_list), 2))

[('', '自然'),
 ('自然', '言語'),
 ('言語', '処理'),
 ('処理', 'は'),
 ('は', 'とても'),
 ('とても', '面白い'),
 ('面白い', '分野'),
 ('分野', 'です'),
 ('です', '。'),
 ('。', '多く'),
 ('多く', 'の'),
 ('の', '応用'),
 ('応用', 'が'),
 ('が', 'あり'),
 ('あり', '、'),
 ('、', '可能'),
 ('可能', '性'),
 ('性', 'は'),
 ('は', '無限'),
 ('無限', 'です'),
 ('です', '。'),
 ('。', '')]

## 問題4-3. 前処理・形態素解析・Word2Vec・cos類似度 (応用)
1. 以下のサンプルデータを用いたdataframeを作成してください。
```
# サンプルの日本語製品レビューのデータ
data = {
    'review_id': [1, 2, 3, 4, 5],
    'review_text': [
        "この商品が大好きです。とても使いやすいです。",
        "最悪の商品です。絶対にお勧めしません。",
        "価格の割に良い品質です。満足しています。",
        "もう一度購入したいと思います。とても良いです。",
        "耐久性がありません。すぐに壊れてしまいました。"
    ]
df = pd.DataFrame(data)
}
```
1. レビュー箇所の前処理としてdef preprocess_text(text: str) -> str:を定義し実行してください。基本的に形態素解析した結果を別カラムに入れる想定です。
1. Word2Vecモデルの学習を実施してください。
1. 各レビューの文書ベクトルを計算し、dataframeの"vector"カラムに入れてください。
1. (応用) 文書ベクトルのリストを取り出してコサイン類似度を計算してください。その結果をindex=df['review_id'], columns=df['review_id']のdataframeに格納してください。


In [184]:
import pandas as pd
# サンプルの日本語製品レビューのデータ
data = {
 'review_id': [1, 2, 3, 4, 5],
 'review_text': [
     "この商品が大好きです。とても使いやすいです。",
     "最悪の商品です。絶対にお勧めしません。",
     "価格の割に良い品質です。満足しています。",
     "もう一度購入したいと思います。とても良いです。",
     "耐久性がありません。すぐに壊れてしまいました。"
 ]}
df = pd.DataFrame(data)
display(df)

,review_id,review_text
0,1,この商品が大好きです。とても使いやすいです。
1,2,最悪の商品です。絶対にお勧めしません。
2,3,価格の割に良い品質です。満足しています。
3,4,もう一度購入したいと思います。とても良いです。
4,5,耐久性がありません。すぐに壊れてしまいました。


In [185]:
# 前処理関数を定義

import MeCab
def preprocess_text(text:str) ->str:


    mecab_tagger = MeCab.Tagger('-Owakati')
    split_text = mecab_tagger.parse(text)
    result = split_text.split()
    return result


# 分割したreviewをデータフレームに追加
df['split_review'] = df['review_text'].apply(lambda x:preprocess_text(x))
display(df)

,review_id,review_text,split_review
0,1,この商品が大好きです。とても使いやすいです。,"[この, 商品, が, 大好き, です, 。, とても, 使い, やすい, です, 。]"
1,2,最悪の商品です。絶対にお勧めしません。,"[最悪, の, 商品, です, 。, 絶対, に, お, 勧め, し, ませ, ん, 。]"
2,3,価格の割に良い品質です。満足しています。,"[価格, の, 割, に, 良い, 品質, です, 。, 満足, し, て, い, ます, 。]"
3,4,もう一度購入したいと思います。とても良いです。,"[もう, 一, 度, 購入, し, たい, と, 思い, ます, 。, とても, 良い, で..."
4,5,耐久性がありません。すぐに壊れてしまいました。,"[耐久, 性, が, あり, ませ, ん, 。, すぐ, に, 壊れ, て, しまい, まし..."


In [186]:
!pip install gensim

In [187]:
# Word2Vecモデルの学習
from gensim.models import Word2Vec

corpus = df['split_review']
print(corpus)
model = Word2Vec(corpus, min_count=1)

# 文書ベクトルを計算
df['vector'] = df['split_review'].apply(lambda x: model.wv[x])

display(df)

0         [この, 商品, が, 大好き, です, 。, とても, 使い, やすい, です, 。]
1        [最悪, の, 商品, です, 。, 絶対, に, お, 勧め, し, ませ, ん, 。]
2     [価格, の, 割, に, 良い, 品質, です, 。, 満足, し, て, い, ます, 。]
3    [もう, 一, 度, 購入, し, たい, と, 思い, ます, 。, とても, 良い, で...
4    [耐久, 性, が, あり, ませ, ん, 。, すぐ, に, 壊れ, て, しまい, まし...
Name: split_review, dtype: object


,review_id,review_text,split_review,vector
0,1,この商品が大好きです。とても使いやすいです。,"[この, 商品, が, 大好き, です, 。, とても, 使い, やすい, です, 。]","[[-0.004794074, -0.0043165493, -0.0048005986, ..."
1,2,最悪の商品です。絶対にお勧めしません。,"[最悪, の, 商品, です, 。, 絶対, に, お, 勧め, し, ませ, ん, 。]","[[0.0025810655, 0.0008398353, -0.0025284812, 0..."
2,3,価格の割に良い品質です。満足しています。,"[価格, の, 割, に, 良い, 品質, です, 。, 満足, し, て, い, ます, 。]","[[-0.009502697, 0.0095598735, -0.0077676703, -..."
3,4,もう一度購入したいと思います。とても良いです。,"[もう, 一, 度, 購入, し, たい, と, 思い, ます, 。, とても, 良い, で...","[[0.005391703, 0.009807461, -0.0070474, -0.000..."
4,5,耐久性がありません。すぐに壊れてしまいました。,"[耐久, 性, が, あり, ませ, ん, 。, すぐ, に, 壊れ, て, しまい, まし...","[[-0.0042715413, -0.009337545, -0.0018549208, ..."


In [188]:
# コサイン類似度を計算
from sklearn.metrics.pairwise import cosine_similarity

for id, vector in zip(df['review_id'],df['vector']):
    cosine_list = []

    for id_2, vector_2 in zip(df['review_id'],df['vector']):
        cosine = cosine_similarity(vector, vector_2)
        cosine_list.append(cosine)

    df[id] = cosine_list

display(df)

,review_id,review_text,split_review,vector,1,2,3,4,5
0,1,この商品が大好きです。とても使いやすいです。,"[この, 商品, が, 大好き, です, 。, とても, 使い, やすい, です, 。]","[[-0.004794074, -0.0043165493, -0.0048005986, ...","[[0.99999994, 0.12152566, -0.15900749, -0.2293...","[[-0.08354691, -0.087274306, 0.11189424, -0.00...","[[0.027882272, -0.07623884, -0.16292438, -0.04...","[[-0.15671083, -0.08196898, -0.03417998, 0.072...","[[0.010187358, 0.06315633, -0.03671494, -0.104..."
1,2,最悪の商品です。絶対にお勧めしません。,"[最悪, の, 商品, です, 。, 絶対, に, お, 勧め, し, ませ, ん, 。]","[[0.0025810655, 0.0008398353, -0.0025284812, 0...","[[-0.0835469, 0.032909956, 0.12152571, 0.11788...","[[0.99999994, 0.097808056, -0.087274306, -0.01...","[[-0.06354572, 0.044037912, -0.07623884, 0.137...","[[0.085711524, -0.033390213, -0.08196898, -0.2...","[[-0.09471882, -0.13388531, 0.06315633, -0.095..."
2,3,価格の割に良い品質です。満足しています。,"[価格, の, 割, に, 良い, 品質, です, 。, 満足, し, て, い, ます, 。]","[[-0.009502697, 0.0095598735, -0.0077676703, -...","[[0.027882287, 0.032909956, -0.035907254, -0.0...","[[-0.06354572, 0.097808056, 0.046538662, 0.031...","[[1.0000001, 0.044037912, 0.045680657, 0.00669...","[[-0.088276416, -0.033390213, -0.055299044, 0....","[[0.0149766365, -0.13388531, 0.1366287, 0.0476..."
3,4,もう一度購入したいと思います。とても良いです。,"[もう, 一, 度, 購入, し, たい, と, 思い, ます, 。, とても, 良い, で...","[[0.005391703, 0.009807461, -0.0070474, -0.000...","[[-0.1567108, 0.25976175, 0.004644075, 0.00242...","[[0.085711524, 0.011469692, 0.07370952, -0.102...","[[-0.088276416, -0.18591596, -0.17210351, 0.04...","[[0.9999998, -0.032136507, 0.124627136, 0.0012...","[[-0.12415385, -0.10168525, -0.025723357, -0.1..."
4,5,耐久性がありません。すぐに壊れてしまいました。,"[耐久, 性, が, あり, ませ, ん, 。, すぐ, に, 壊れ, て, しまい, まし...","[[-0.0042715413, -0.009337545, -0.0018549208, ...","[[0.010187358, -0.016604405, -0.1590075, 0.259...","[[-0.09471882, -0.142324, 0.11189424, 0.086806...","[[0.0149766365, 0.008393042, -0.16292438, -0.1...","[[-0.12415385, 0.08020813, -0.03417998, -0.014...","[[1.0000001, -0.059548575, -0.03671494, 0.0127..."


#問題4-4. TF-IDF
1. 以下のデータを利用してdataframeを作成してください。


```
data = {
    'review_id': [1, 2, 3, 4, 5],
    'review_text': [
        "この商品が大好きです。とても使いやすいです。",
        "最悪の商品です。絶対にお勧めしません。",
        "価格の割に良い品質です。満足しています。",
        "もう一度購入したいと思います。とても良いです。",
        "耐久性がありません。すぐに壊れてしまいました。"
    ]
}

# データフレームの作成と保存
df = pd.DataFrame(data)
```
1. このdataframeを用いて各レビューでのtf-idfを算出してdataframeの別カラムに格納してください。
1. tf-idfのdataframeを作成してください。index=review_id, カラム: 分割した単語, 値: tf-idf値


In [189]:
# データの読み込み
data = {
    'review_id': [1, 2, 3, 4, 5],
    'review_text': [
        "この商品が大好きです。とても使いやすいです。",
        "最悪の商品です。絶対にお勧めしません。",
        "価格の割に良い品質です。満足しています。",
        "もう一度購入したいと思います。とても良いです。",
        "耐久性がありません。すぐに壊れてしまいました。"
    ]
}

# データフレームの作成と保存
df = pd.DataFrame(data)

In [190]:
import MeCab
def preprocess_text(text:str) ->str:


    mecab_tagger = MeCab.Tagger('-Owakati')
    split_text = mecab_tagger.parse(text)
    result = split_text.strip()
    return result

# 分割したreviewをデータフレームに追加
df['split_review'] = df['review_text'].apply(lambda x:preprocess_text(x))
display(df)

,review_id,review_text,split_review
0,1,この商品が大好きです。とても使いやすいです。,この 商品 が 大好き です 。 とても 使い やすい です 。
1,2,最悪の商品です。絶対にお勧めしません。,最悪 の 商品 です 。 絶対 に お 勧め し ませ ん 。
2,3,価格の割に良い品質です。満足しています。,価格 の 割 に 良い 品質 です 。 満足 し て い ます 。
3,4,もう一度購入したいと思います。とても良いです。,もう 一 度 購入 し たい と 思い ます 。 とても 良い です 。
4,5,耐久性がありません。すぐに壊れてしまいました。,耐久 性 が あり ませ ん 。 すぐ に 壊れ て しまい まし た 。


In [191]:
from sklearn.feature_extraction.text import TfidfVectorizer

# tfidfモデルの作成と学習
model = TfidfVectorizer()
corpus = df['split_review'].tolist()
print(corpus)

model.fit(corpus)

['この 商品 が 大好き です 。 とても 使い やすい です 。', '最悪 の 商品 です 。 絶対 に お 勧め し ませ ん 。', '価格 の 割 に 良い 品質 です 。 満足 し て い ます 。', 'もう 一 度 購入 し たい と 思い ます 。 とても 良い です 。', '耐久 性 が あり ませ ん 。 すぐ に 壊れ て しまい まし た 。']


TfidfVectorizer()

In [194]:
# tfidfカラムの追加
df['tfidf'] = df['split_review'].apply(lambda x: model.transform([x]).toarray()[0])
display(df)

import numpy as np
x = df['split_review'].apply(lambda x: model.transform([x]).toarray()[0])
y = list(x)
data = model.get_feature_names_out()
tfidf = pd.DataFrame(x.values)
display(tfidf)

,review_id,review_text,split_review,tfidf
0,1,この商品が大好きです。とても使いやすいです。,この 商品 が 大好き です 。 とても 使い やすい です 。,"[0.0, 0.39009465354125766, 0.0, 0.0, 0.0, 0.43..."
1,2,最悪の商品です。絶対にお勧めしません。,最悪 の 商品 です 。 絶対 に お 勧め し ませ ん 。,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.26213107330293306,..."
2,3,価格の割に良い品質です。満足しています。,価格 の 割 に 良い 品質 です 。 満足 し て い ます 。,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.26213107330293306,..."
3,4,もう一度購入したいと思います。とても良いです。,もう 一 度 購入 し たい と 思い ます 。 とても 良い です 。,"[0.0, 0.0, 0.0, 0.0, 0.3993568140170139, 0.224..."
4,5,耐久性がありません。すぐに壊れてしまいました。,耐久 性 が あり ませ ん 。 すぐ に 壊れ て しまい まし た 。,"[0.38775666010579296, 0.0, 0.38775666010579296..."


,0
0,"[0.0, 0.39009465354125766, 0.0, 0.0, 0.0, 0.43..."
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.26213107330293306,..."
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.26213107330293306,..."
3,"[0.0, 0.0, 0.0, 0.0, 0.3993568140170139, 0.224..."
4,"[0.38775666010579296, 0.0, 0.38775666010579296..."
